In [57]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
import datetime as dt

In [ ]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
data = pd.read_csv(url)
data = data[data["continent"].notna()]
data['date'] = pd.to_datetime(data['date'])
data[["new_cases", "total_cases","total_cases_per_million"]] = data[[
    "new_cases", "total_cases","total_cases_per_million"]].fillna(0)

In [107]:
dates = pd.date_range(start="2020-01-03",end=pd.Timestamp.today())
marks = pd.date_range(start="2020-01-03",end=pd.Timestamp.today(),freq="MS")

In [151]:
continents= data["continent"].unique().tolist()
continents.append("World")
continents.remove("Oceania")

In [166]:


app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(children='Térkép', style={'textAlign': 'center', 'fontSize': 30}),
    html.Hr(),
    dcc.Dropdown(id='dropdown_col',
                options=[
                    {'label': 'Total cases', 'value': 'total_cases'},
                    {'label': 'Total deaths', 'value': 'total_deaths'},
                    {'label': 'Total cases per million people', 'value': 'total_cases_per_million'},
                    {'label': 'Total deaths per million people', 'value': 'total_deaths_per_million'}],
                value='total_cases'),
    dcc.Dropdown(id='dropdown_cont',options=continents,
                 value="World"),
    dcc.Slider(min=0,
               max=len(marks),
               value=0,
               marks={i : str(yearmon) for i, yearmon in enumerate(marks.strftime("%Y-%m"))},
               step=1,
               id='dateslider'),
    dcc.Graph(id='graph_sc_geo',figure={}),
])


@callback(
    Output('graph_sc_geo', 'figure'),
    Input('dropdown_col', 'value'),
    Input('dateslider','value'),
    Input('dropdown_cont','value')
)
def update_graph(col,month,cont):
    df = data[data["date"]==marks[month]] 
    fig = px.scatter_geo(df,
                         locations="iso_code",
                         size=col,
                         scope=cont.lower(),
                         size_max=40,
                         height=800,
                         color="continent"
                         )
    fig.update_geos(showcountries=True)
    return fig

                                                 
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

